In [1]:
import numpy as np
import pandas as pd
import os

# Reproducing numbers from Iwata et al paper

## AES using 2-hole population analysis
M. Mitani et al, J. El. Spec. and Rel. Phenom., 128, 2003, 103-117
https://www.sciencedirect.com/science/article/pii/S0368204802002700

Essentially, we'll take the data from Table 3 and use them to reproduce Table 4


In [2]:
# Data from Table 2, column O*(L)
# indexing from 0, MOs - 1a1, 2a1, 1b2, 3a1, 1b1 
lowdin_core_vdz = np.array([0.99, 1.57, 1.61, 1.89, 1.93], dtype = np.float64)
lowdin_core_vtz = np.array([0.98, 1.33, 1.55, 1.86, 1.88], dtype = np.float64)

In [3]:
# Now let's try to reproduce column 4 from Table 3
K = 1 / np.sqrt(6.993799999999999) # so that 1B1 state has I = 1.0

# Eq. 7
def t_vw_singlet(pop, v, w, K):
    if v == w:
        return K * pop[v]
    else:
        return K * (pop[v] + pop[w]) / np.sqrt(2)
    
def t_vw_triplet(pop, v, w, K):
    return K * (pop[v] - pop[w]) * np.sqrt(3/2.)

def t_vw(MOpop, v, w, K, spinmult):
    if spinmult in (1, 's'):
        return t_vw_singlet(MOpop, v, w, K)
    elif spinmult in (3, 't'):
        return t_vw_triplet(MOpop, v, w, K)
    else:
        print("Invalid multiplicity", spinmult)
        raise

## Approximate computation - single CSF final states

Assuming that final states are described by a single CSF which is generally true for a single water molecule according to Table 4.

In [4]:
# Let's just assume single configuration final states
I_approx = []
# (v, w, 's'|'t')
states = [(3, 4, 't'), # tB1
          (4, 4, 's'), # sA1
          (3, 4, 's'), # 
          (2, 3, 't'),
          (3, 3, 's'),
          (2, 4, 't'),
          (2, 4, 's'),
          (2, 3, 's'),
          (2, 2, 's'),
          (1, 4, 't'),
          (1, 3, 't'),
          (1, 2, 't'),
          (1, 4, 's'),
          (1, 3, 's'), # sA1
          (1, 2, 's'), # sB2
          ###...last state (2a_1)^(-2)
          (1, 1, 's') # sA1
         ]

for state in states:
    if state[2] == 't':
        t = t_vw_triplet(lowdin_core_vtz, state[0], state[1], K)
        I_approx.append(t**2)
    else:
        t = t_vw_singlet(lowdin_core_vtz, state[0], state[1], K)
        I_approx.append(t**2)

# Here are the final numbers for comparison with the paper
I_paper = np.array([0.0, 0.5, 1.0, 0.02, 0.5, 0.02, 0.84, 0.83, 0.35, 0.06, 0.06, 0.01, 0.74, 0.72, 0.59, 0.26])
print("  I   I(paper)")
for i in range(len(I_paper)):
    print("%.3f %.2f" % (I_approx[i], I_paper[i]))

  I   I(paper)
0.000 0.00
0.505 0.50
1.000 1.00
0.021 0.02
0.495 0.50
0.023 0.02
0.841 0.84
0.831 0.83
0.344 0.35
0.065 0.06
0.060 0.06
0.010 0.01
0.737 0.74
0.728 0.72
0.593 0.59
0.253 0.26


## Comparison with FANO-CI data

In [5]:
# Now let's see our FANO-CI for data
# Note that the ordering is different, first singlet states, then triplet states
GAMMA_MOM = '../h2o/core/CC-PVTZ/scr-fanoci-1s-mom-5spd/gammas_avg.dat'
GAMMA_NEUTRAL = '../h2o/core/CC-PVTZ/scr-fanoci-1s-neutral-5spd/partial_gammas.dat'
gammas_fanoci_mom = pd.read_csv(GAMMA_MOM, comment="#", delim_whitespace=True)
gammas_fanoci_neutral = pd.read_csv(GAMMA_NEUTRAL, comment="#", delim_whitespace=True)
# Let's append info about spin multiplicity just to keep things organized
nsinglets = 10
gammas_fanoci_neutral['SpinMult'] = 1; gammas_fanoci_mom['SpinMult'] = 1;
gammas_fanoci_neutral.loc[nsinglets:, 'SpinMult'] = 3; gammas_fanoci_mom.loc[nsinglets:, 'SpinMult'] = 3;
print(gammas_fanoci_neutral)
print(gammas_fanoci_mom)

       Gamma       Std  SpinMult
0   10.36380  1.098000         1
1    7.58593  0.285780         1
2    5.50547  0.595272         1
3    4.30086  0.518590         1
4    5.39115  0.297638         1
5    4.37881  0.487662         1
6    7.77136  0.272843         1
7    9.05474  0.220602         1
8    4.45675  0.151596         1
9    7.65108  0.126462         1
10   0.00000  0.000000         3
11   0.00000  0.000000         3
12   0.00000  0.000000         3
13   2.09275  0.158422         3
14   0.00000  0.000000         3
15   0.00000  0.000000         3
       Gamma       Std  SpinMult
0   27.97370  2.593690         1
1   17.11170  1.777560         1
2   13.00930  1.288280         1
3   14.50620  1.826940         1
4   12.38000  0.530065         1
5   12.61540  1.833580         1
6   20.51730  2.067110         1
7   21.34910  0.431274         1
8   13.39800  0.680975         1
9   16.29560  0.278962         1
10   7.65049  2.191840         3
11   6.12153  0.580913         3
12   0.000

In [6]:
# The intensity is directly proportional to Gammas so we'll just normalize on the second singlet
I_fano_MOM = np.array(gammas_fanoci_mom['Gamma']) / gammas_fanoci_mom['Gamma'][1]
print(I_fano_MOM)

[1.63477036 1.         0.7602576  0.84773576 0.72348159 0.73723826
 1.19902172 1.24763174 0.78297305 0.95230749 0.44709117 0.35773944
 0.         0.33266069 0.         0.        ]


**We can see that the MOM based FANOCI intensities are quite different!**

For example, the first singlet state is more intense than the first one. Also, the triplets have much higher intensity.
However, the intensities based on densities instead of populations,fifth column in Tab.4, the triplets have higher intensity as well.

In [7]:
I_fano_neutral = np.array(gammas_fanoci_neutral['Gamma']) / gammas_fanoci_neutral['Gamma'][1]
print(I_fano_neutral)

[1.36618714 1.         0.72574754 0.56695224 0.71067753 0.57722784
 1.02444394 1.19362293 0.58750213 1.00858827 0.         0.
 0.         0.27587257 0.         0.        ]


**The FANO intensities from neutral WF generally follow the MOM based results!**

Note that appart from one, stieltjes was not converging for the triplet states. Some of them probably because the intensity was zero. However, the first triplet state should have some intensity according to multiple source. I'll need to ask Tsveta whether the Stieltjes convergence could be improved.

## Full 2hpop calculation
Let's try the full calculation including the CI expansion. 
We'll focus on third singlet state $^1A_1$, i.e. the fifth row of Tab. 4
The dominant configuration with CI coeff=0.93 is $(3\mathrm{a}_1)^{-2}$
The paper does not give the rest of CI expansion so we'll take it from FANO-CI data.

In [8]:
# Just a little structer to hold CI data for a single final 2h state
class CI_2h_state:
    
    def __init__(self, energy):
        self.CSFs = []
        self.energy = energy
            
    def setMultiplicity(self, mult):
        if mult in ('s', 'I>'):
            self.spinmult = 1
        elif mult in ('t', 'III>'):
            self.spinmult = 3
        else:
            self.spinmult = int(mult)
        
    def addCSF(self, i, j, Cij):
        self.CSFs.append((i, j, Cij))
        
    def getCSFs(self):
        return self.CSFs
    
    def print_state(self):
        print("SpinMult = %d Energy = %.5f" % (self.spinmult, self.energy))
        for CSF in self.CSFs:
            print("% .5f | %d %d >" % (CSF[2], CSF[0], CSF[1]))



def read_fanoci_states(fname):
    CI_states = []
    with open(fname, 'r') as f:
        nstates = 3
        for line in f:
            l = line.split()
            #if len(l) == 2 and l[0].strip() == '%d:'% (nstates+1):
            #    break
            if len(l) == 2:
                # We're reading a new state!
                energy = float(l[1])
                CI_states.append(CI_2h_state(energy))
            elif len(l) > 2:
                # Convert to zero-based states
                v = int(l[1]) - 1
                w = int(l[2][0:-1]) - 1
                spinmult = l[3]
                Cvw  = float(l[4])
                CI_states[-1].addCSF(v, w, Cvw)
                CI_states[-1].setMultiplicity(spinmult)
               
    return CI_states

In [9]:
NEUTRAL_CONFS_FILE = '../h2o/core/CC-PVTZ/scr-fanoci-1s-neutral-5spd/states.2h.txt'
MOM_CONFS_FILE = '../h2o/core/CC-PVTZ/scr-fanoci-1s-mom-5spd-reordered/states.2h.txt'
# Let's compare states calculated from neutral RHF WF and core-ionized ROHF MOM
# Data in Table 4 are for core hole
# I am not sure I currently trust the FANOCI data for MOM WF
neutral_final_states = read_fanoci_states(NEUTRAL_CONFS_FILE)
mom_final_states = read_fanoci_states(MOM_CONFS_FILE)

In [10]:
# Compare the MOM and neutral 2h states
i = 2
neutral_final_states[i].print_state()
print()
mom_final_states[i].print_state()

SpinMult = 1 Energy = 1.85268
 0.00201 | 1 2 >
-0.11929 | 1 3 >
 0.01527 | 2 3 >
 0.07882 | 1 1 >
 0.15481 | 2 2 >
-0.95599 | 3 3 >
-0.20354 | 4 4 >

SpinMult = 1 Energy = 3.24589
-0.00213 | 1 4 >
-0.99986 | 2 4 >
 0.01654 | 3 4 >


---
We can see that the MOM WF looks of C($3a_1)^{-2}$) = 0.9998 instead of 0.93 as in the paper. It's not clear where does the discrepancy comes from.
The neutral WF has C($3a_1)^{-2}$) = 0.95, which is closer to the paper, so we'll work with it from now on.

Let's calculate the intensity according to eq. 14 without the cross terms (which is what they do in the paper).

We'll need the Lowdin population data from earlier.

In [11]:
# We also need the second singlet state to compare intensities
# In table 4 this state has I = 1 so we'll set the K accordingly
def aes_intensity_2hpop(MOpop, final_state):
    K = 1 / np.sqrt(6.993799999999999) / np.sqrt(0.9986806048781381)
    I = 0.0
    mult = final_state.spinmult
    for csf in final_state.getCSFs():
        v = csf[0]
        w = csf[1]
        Cvw = csf[2]
        t = t_vw(pop, v, w, K, mult)
        I += Cvw**2 * t**2
    return I

pop = lowdin_core_vtz
I2 = aes_intensity_2hpop(pop, neutral_final_states[1])
I3 = aes_intensity_2hpop(pop, neutral_final_states[2])
print(I2, I3)

1.0 0.4940295343235541


---
Great! The result in Tab. 4 for third singlet state is 0.50, so we're a little of, but that's because we have slightly different CI coefficients.

Now let's do all states!
Note that the ordering is given by FANOCI: first singlets, then triplets

In [12]:
MOpop = lowdin_core_vtz
i = 0
paper2fano_indeces = (1, 2, 4, 6, 7, 8, 12, 13, 14, 15, 0, 3, 5, 9, 10, 11)
print('I     I(paper)')
for state in neutral_final_states:
    I = aes_intensity_2hpop(MOpop, state)
    print("%.3f %.2f" % (I, I_paper[paper2fano_indeces[i]]))
    i += 1

I     I(paper)
0.505 0.50
1.000 1.00
0.494 0.50
0.842 0.84
0.832 0.83
0.346 0.35
0.739 0.74
0.725 0.72
0.594 0.59
0.258 0.26
0.000 0.00
0.023 0.02
0.021 0.02
0.065 0.06
0.060 0.06
0.010 0.01
